In [ ]:
!pip install -q gigachat

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from gigachat import GigaChat
from google.colab import userdata

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
def load_model(model_name: str):
    model_name = model_name.lower()
    hf_models = ('qwen', 'gpt')
    if model_name not in hf_models and model_name != 'gigachat':
        raise NameError(f'No model named {model_name} available!\nPlease, use one of the available models: ' + ', '.join(hf_models) + ', gigachat')

    if model_name in hf_models:
        if model_name == 'qwen':
            model_name = '/kaggle/input/qwen2.5/transformers/3b-instruct/1'
    
        else:
            model_name = 'gpt2-xl'

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        return model, tokenizer
    
    else:
        token = userdata.get('GigaToken') # Change to Kaggle Secrets
        client = GigaChat(credentials=token, verify_ssl_certs=False)
        return client
        

In [3]:
def generate_text_hf(
    text: str,
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    max_length: int = 100
):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )

In [8]:
model, tokenizer = load_model("gpt")

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
generated = generate_text_hf(
    "This is a test launch. Can you hear me?",
    model,
    tokenizer,
    max_length=200
)

print(generated)

 Okay. Okay, let 'em hear me. Let me hear you, okay? Okay, you know what? I'm sorry, I think I killed their communication. I think I messed up and I'm sorry, but I tried to send this message to your friends and that message came back as 'Sorry, could not receive. We can hear you now.' Can you hear me? Okay. Okay, I got the message. I believe this is real voiceover, thank you.

[COMMERCIAL BREAK]

HANNITY: We're also joined now by another, Michael Grynbaum, author of "Secret Wars: The CIA, the Pentagon and the Rise of America's Secret Government," and author of the book, "The CIA Secrets."

Let's begin with the question posed in the title, does this test launch have any meaning?

MICHAEL GRYNTBAUM,
